In [25]:
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlalchemy import text

In [4]:

# Cargar las variables de entorno desde .env
load_dotenv()

DB_USER = os.getenv("DB_USER")  # Puedes agregar esto al .env si prefieres
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = "spaceman"

# Crear la URL de conexión a PostgreSQL
database_url = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

# Crear una conexión a la base de datos usando SQLAlchemy
engine = create_engine(database_url)

In [5]:
# Listado de tablas a extraer
tablas = ["categorias", "productos", "secciones", "sectores", "subcategorias", "tickets"]

# Diccionario para almacenar los DataFrames
dfs = {}

try:
    with engine.connect() as connection:
        for tabla in tablas:
            print(f" Extrayendo tabla: {tabla} ...")
            query = f"SELECT * FROM maestros.{tabla}"  # Asegúrate de que el esquema es 'maestros'
            dfs[tabla] = pd.read_sql(query, con=connection)

    print("Tablas extraídas correctamente.")
except Exception as e:
    print(f"Error al extraer las tablas: {e}")
finally:
    engine.dispose()  # Cierra la conexión

 Extrayendo tabla: categorias ...
 Extrayendo tabla: productos ...
 Extrayendo tabla: secciones ...
 Extrayendo tabla: sectores ...
 Extrayendo tabla: subcategorias ...
 Extrayendo tabla: tickets ...
Tablas extraídas correctamente.


In [9]:
Vista_1=pd.merge(dfs['tickets'],dfs['productos'],how='inner',on=['eancode','idcadena']).merge(dfs['categorias'],how='inner',on=['id_categoria','idcadena'])

In [10]:
Vista_1=Vista_1[Vista_1['anulado']==False]

In [16]:
Vista_1_final=Vista_1.groupby(['id_categoria','idcadena']).agg(
    Categoria=pd.NamedAgg(column='categoria',aggfunc='last'),
    Total_unidades_vendidas=pd.NamedAgg(column='unidades_vendidas',aggfunc='sum')
).reset_index()

In [19]:
dfs['tickets']['Facturacion']=dfs['tickets']['unidades_vendidas']*dfs['tickets']['precio_regular']

In [27]:
vista_2_final = dfs['tickets'][dfs['tickets']['anulado'] == False].groupby('fecha').agg(
    Facturacion=pd.NamedAgg(column='Facturacion', aggfunc='sum'),
    Total_vendidos=pd.NamedAgg(column='unidades_vendidas', aggfunc='sum'),
    Total_tickets=pd.NamedAgg(column='ticket', aggfunc='nunique')
).reset_index()

In [33]:
try:
    with engine.connect() as connection:
        # Subir los DataFrames como tablas temporales
        Vista_1_final.to_sql('tmp_ventas_categoria', con=connection, schema='maestros', if_exists='replace', index=False)
        vista_2_final.to_sql('tmp_ventas_dias', con=connection, schema='maestros', if_exists='replace', index=False)

        # Crear vistas en PostgreSQL
        sql_views = """
        CREATE OR REPLACE VIEW maestros.ventas_por_categoria_python AS
        SELECT * FROM maestros.tmp_ventas_categoria;

        CREATE OR REPLACE VIEW maestros.ventas_dias_python AS
        SELECT * FROM maestros.tmp_ventas_dias;
        """
        connection.execute(text(sql_views))
        connection.commit()

    print("Vistas creadas correctamente en PostgreSQL.")
except Exception as e:
    print(f"Error al crear las vistas: {e}")
finally:
    engine.dispose()

Vistas creadas correctamente en PostgreSQL.
